In [ ]:
import re
import pandas as pd

def create_dataframe_from_file(filename):
    """
    Reads a file with a specific format and parses it into a Pandas DataFrame.

    The function assumes documents are separated by '*TEXT' and the header
    is separated from the article by the first double newline.

    Args:
        filename (str): The path to the input file (e.g., 'TIME.ALL').

    Returns:
        pandas.DataFrame: A DataFrame containing the parsed data with columns
                          [Index, Date, Page, Article]. Returns an empty
                          DataFrame on error.
    """

    try:
        with open(filename, 'r') as f:
            content = f.read()
    except FileNotFoundError:
        print(f"Error: The file '{filename}' was not found.")
        return pd.DataFrame()

    # Split the file content into documents based on the '*TEXT' delimiter
    # The (?=...) is a positive lookahead to keep the delimiter
    documents = re.split(r'(?=\*TEXT)', content)

    # This list will hold a dictionary for each document (row)
    data_for_df = []

    # Process each document block
    for doc_string in documents:
        doc_string = doc_string.strip()
        if not doc_string:
            continue

        # The key step: split the document at the *first* double newline.
        # This separates the header from the main article text.
        parts = doc_string.split('\n\n', 1)
        if len(parts) != 2:
            # Skip any malformed entries that don't have a header and a body
            print(f"Skipping malformed document block: {doc_string[:50]}...")
            continue

        header, article_text = parts

        # Now, parse the header line
        header_parts = header.split()

        # Expected header format: ['*TEXT', '017', '01/04/63', 'PAGE', '020']
        try:
            doc_id = header_parts[1]
            doc_date = header_parts[2]
            # Find the index of 'PAGE' to robustly get the page number
            page_index = header_parts.index('PAGE')
            doc_page = int(header_parts[page_index + 1])

            # Create a dictionary for this document's data
            row_data = {
                'Index': doc_id,
                'Date': doc_date,
                'Page': doc_page,
                'Article': article_text.strip() # Clean up whitespace
            }
            data_for_df.append(row_data)

        except (IndexError, ValueError) as e:
            # Handle cases where the header is not in the expected format
            print(f"Skipping document with malformed header: '{header}'. Error: {e}")
            continue

    if not data_for_df:
        print("No valid documents were found to create a DataFrame.")
        return pd.DataFrame()

    # Create the DataFrame from our list of dictionaries
    df = pd.DataFrame(data_for_df)

    # --- Post-processing the DataFrame ---

    # Set the 'Index' column as the actual DataFrame index
    df.set_index('Index', inplace=True)

    # Convert the 'Date' column to a proper datetime object for easier analysis
    # The format '%m/%d/%y' tells pandas how to interpret '63' as '1963'
    #df['Date'] = pd.to_datetime(df['Date'], format='%m/%d/%y')

    # The 'Page' column is already an integer from our parsing step

    return df

def create_inverted_index_from_df(df):
    """
    Creates an inverted index and a vocabulary with document frequencies
    from a given DataFrame.

    Args:
        df (pd.DataFrame): A DataFrame with the document ID as its index
                           and a column named 'Article' containing the text.

    Returns:
        tuple: A tuple containing two dictionaries:
               1. inverted_index (dict): Maps terms to a sorted list of doc IDs.
               2. vocab_df (dict): Maps terms to their document frequency.
    """
    inverted_index = {}

    # df.iterrows() yields the index (our doc_id) and the row data
    for doc_id, row in df.iterrows():
        article_text = row['Article']

        # --- Text Preprocessing ---
        # 1. Convert to lowercase
        article_text = article_text.lower()
        # 2. Remove punctuation and numbers
        article_text = re.sub(r'[^a-z\s]', ' ', article_text)
        # 3. Tokenize into words
        words = article_text.split()

        # Get unique words to handle document frequency correctly
        unique_words = set(words)

        # --- Update the Inverted Index ---
        for term in unique_words:
            # If term is not in the index, initialize its list
            if term not in inverted_index:
                inverted_index[term] = []
            # Append the current document ID
            inverted_index[term].append(doc_id)

    # The doc IDs are naturally added in sorted order because we iterate through
    # the DataFrame row by row. No extra sorting of the lists is needed.

    # --- Create Vocabulary with Document Frequency ---
    # The document frequency is the length of the postings list (the list of doc IDs)
    vocab_df = {term: len(postings) for term, postings in inverted_index.items()}

    return inverted_index, vocab_df

def execute_query(query, inverted_index, df):
    """
    Executes a boolean query (AND, OR) against the inverted index.

    Args:
        query (str): The boolean query string (e.g., "term1 AND term2").
        inverted_index (dict): The inverted index dictionary.
        df (pd.DataFrame): The main DataFrame to retrieve final documents.

    Returns:
        pd.DataFrame: A DataFrame containing the documents that match the query.
    """
    # Helper to get postings for a term, returns an empty list if term not found
    def get_postings(term):
        return inverted_index.get(term, [])

    # Preprocess the query: lowercase and split into components
    query_parts = query.lower().split()

    if not query_parts:
        print("Query is empty.")
        return pd.DataFrame()

    # Start with the results for the first term
    # We use a set for efficient intersection and union operations
    result_set = set(get_postings(query_parts[0]))

    # Process the rest of the query, which should be in pairs of (operator, term)
    # This evaluates the query from left to right.
    i = 1

    while i < len(query_parts):
        operator = query_parts[i]

        # Check for a malformed query (operator without a term)
        if i + 1 >= len(query_parts):
            print(f"Error: Malformed query. Operator '{operator}' is not followed by a term.")
            return pd.DataFrame()

        term = query_parts[i+1]
        term_postings = set(get_postings(term))

        if operator == 'and':
            # INTERSECTION: Keep only documents present in both sets
            result_set = result_set.intersection(term_postings)
        elif operator == 'or':
            # UNION: Keep documents present in either set
            result_set = result_set.union(term_postings)
        else:
            print(f"Error: Unsupported operator '{operator}'. Only 'and'/'or' are supported.")
            return pd.DataFrame()

        # Move to the next operator
        i += 2

    # If the result set is empty, return an empty DataFrame
    if not result_set:
        return pd.DataFrame()

    # Retrieve the full documents from the main DataFrame using the final list of doc IDs
    # We convert the set to a list for .loc indexing
    final_docs = list(result_set)
    # Sort the results to maintain a consistent order
    final_docs.sort()

    return df.loc[final_docs]


# --- Main execution block ---
if __name__ == "__main__":
    file_path = 'TIME.ALL'

    # --- Step 1: Create the DataFrame ---
    time_df = create_dataframe_from_file(file_path)

    if not time_df.empty:
        print(time_df)

        # --- Step 2: Generate the Inverted Index from the DataFrame ---
        inverted_index, vocabulary_df = create_inverted_index_from_df(time_df)

        print("\n--- Inverted Index (Postings Lists) ---")
        print(inverted_index)
        print(vocabulary_df)

        """
        # Print a few examples from the inverted index
        print(f"de: {inverted_index.get('de')}")
        # Print the same examples for the vocabulary
        print(f"de: {vocabulary_df.get('de')}")"""

        print(f"\nTotal unique terms (vocabulary size): {len(vocabulary_df)}")

        print("\n")
        print("="*60)
        print("---EXECUTING BOOLEAN QUERIES---")
        print("="*60)

        # Query 1: A simple AND query
        query = "nato AND out AND too"
        print(f"\n>>> Executing Query: '{query}'")
        results1 = execute_query(query, inverted_index, time_df)
        if not results1.empty:
            print(f"Found {len(results1)} matching document(s):")
            print(results1[['Date', 'Page']]) # Display summary of results
        else:
            print("No documents found.")

           Date  Page                                            Article
Index                                                                   
017    01/04/63    20  THE ALLIES AFTER NASSAU IN DECEMBER 1960, THE ...
018    01/04/63    21  RUSSIA WHO'S IN CHARGE HERE ? IT WAS IN 1954 T...
019    01/04/63    21  BERLIN ONE LAST RUN HANS WEIDNER HAD BEEN HOPI...
020    01/04/63    21  THE ROAD TO JAIL IS PAVED WITH NONOBJECTIVE AR...
021    01/04/63    22  THE CONGO ROUND 3 ? THE SOUND OF\n\nCHRISTMAS ...
...         ...   ...                                                ...
559    12/27/63    24  SOUTH VIET NAM     PLEASE SEND ORDERS    IN TH...
560    12/27/63    24  INDIA     SEA LAWYER     EVER SINCE RED CHINA'...
561    12/27/63    24  CAMBODIA     THE SLUMBERING PRINCE     FORMER ...
562    12/27/63    24  SOUTH KOREA FATIGUES TO FLANNELS SEOUL WAS DEC...
563    12/27/63    24  UNITED NATIONS POTENT PYGMY IN MANHATTAN LAST ...

[423 rows x 3 columns]

--- Inverted Index (Postin